In [1]:
%matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg


In [2]:
from mpl_toolkits.basemap import Basemap

In [3]:
from mpl_toolkits.basemap import Basemap    #①
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None, 
width=8E6, height=8E6, lat_0=30, lon_0=120)    #②
m.etopo(scale=0.5, alpha=0.5)    #③

x, y = m(120.7, 31.3)    #④
plt.plot(x, y, 'ok', markersize=5)    #⑤
plt.text(x, y, ' Soochow', fontsize=12)    #⑥

In [4]:
cities = pd.read_csv("/home/qiwsir/Documents/DataAnalysis/chapter03/city_population.csv")
lat = cities['latd'].values
lon = cities['longd'].values
population = cities['population'].values
area = cities['area'].values/10

fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', width=5E5, height=6E5, lat_0=32.9, lon_0=119.2,resolution='h')
m.shadedrelief()    #设置地形图效果
m.drawcoastlines(color='gray')    #描绘海岸线

#画散点图
m.scatter(lon, lat, latlon=True, c=np.log10(population), s=area, cmap='Reds', alpha=0.5)

plt.colorbar(label='log$_{10}$(population)')

for area in [100, 300, 500]:
    plt.scatter([], [], c='k', alpha=0.3, s=area, label=str(area) + ' km$^2$')

plt.legend(scatterpoints=1, frameon=False, labelspacing=1, title='City Area')